In [22]:
import bs4
import urllib 
import pandas as pd
import requests
from tqdm import tqdm

In [38]:
url = "http://pokemondb.net/pokedex/national"
r = requests.get(url)  #urllib.request.urlopen() ne fonctionne pas avec ce site, donc on utilise un autre package
request_text = r.text
page = bs4.BeautifulSoup(request_text, "lxml")

In [39]:
colnames_set = False
colnames = []
data = []
for item in tqdm(page.findAll("div", {'class' : 'infocard'})):
    name = item.find('a', ['ent-name']).getText()
    #print(name)
    url = "https://pokemondb.net"+item.find('a').get("href")
    r = requests.get(url)
    request_text = r.text
    page = bs4.BeautifulSoup(request_text, "lxml")
    
    data_pokemon = []
    for i in range(4):
        #print(i)
        table = page.findAll('table')[i] 
        df = pd.read_html(str(table))[0]
        if not colnames_set: 
            colnames = colnames + list(df[0])
        data_pokemon = data_pokemon + list(df[1])
        
    colnames_set = True
    data.append(data_pokemon)
    
df = pd.DataFrame(data,columns=colnames)

100%|██████████| 809/809 [02:39<00:00,  5.08it/s]


In [40]:
df.head()

,National №,Type,Species,Height,Weight,Abilities,Local №,EV yield,Catch rate,Base Friendship,...,Egg Groups,Gender,Egg cycles,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Total
0,001,Grass Poison,Seed Pokémon,0.7 m (2′04″),6.9 kg (15.2 lbs),1. OvergrowChlorophyll (hidden ability),001 (Red/Blue/Yellow)226 (Gold/Silver/Crystal)...,1 Special Attack,"45 (5.9% with PokéBall, full HP)",70 (normal),...,"Grass, Monster","87.5% male, 12.5% female","20 (4,884–5,140 steps)",45,49,49,65,65,45,318
1,002,Grass Poison,Seed Pokémon,1.0 m (3′03″),13.0 kg (28.7 lbs),1. OvergrowChlorophyll (hidden ability),002 (Red/Blue/Yellow)227 (Gold/Silver/Crystal)...,"1 Special Attack, 1 Special Defense","45 (5.9% with PokéBall, full HP)",70 (normal),...,"Grass, Monster","87.5% male, 12.5% female","20 (4,884–5,140 steps)",60,62,63,80,80,60,405
2,003,Grass Poison,Seed Pokémon,2.0 m (6′07″),100.0 kg (220.5 lbs),1. OvergrowChlorophyll (hidden ability),003 (Red/Blue/Yellow)228 (Gold/Silver/Crystal)...,"2 Special Attack, 1 Special Defense","45 (5.9% with PokéBall, full HP)",70 (normal),...,"Grass, Monster","87.5% male, 12.5% female","20 (4,884–5,140 steps)",80,82,83,100,100,80,525
3,004,Fire,Lizard Pokémon,0.6 m (2′00″),8.5 kg (18.7 lbs),1. BlazeSolar Power (hidden ability),004 (Red/Blue/Yellow)229 (Gold/Silver/Crystal)...,1 Speed,"45 (5.9% with PokéBall, full HP)",70 (normal),...,"Dragon, Monster","87.5% male, 12.5% female","20 (4,884–5,140 steps)",39,52,43,60,50,65,309
4,005,Fire,Flame Pokémon,1.1 m (3′07″),19.0 kg (41.9 lbs),1. BlazeSolar Power (hidden ability),005 (Red/Blue/Yellow)230 (Gold/Silver/Crystal)...,"1 Special Attack, 1 Speed","45 (5.9% with PokéBall, full HP)",70 (normal),...,"Dragon, Monster","87.5% male, 12.5% female","20 (4,884–5,140 steps)",58,64,58,80,65,80,405


# Explications

In [4]:
item = page.findAll("div", {'class' : 'infocard'})[0]
url = "https://pokemondb.net"+item.find('a').get("href")
r = requests.get(url)
request_text = r.text
page = bs4.BeautifulSoup(request_text, "lxml")

In [8]:
page.findAll('table') [2]

<table class="vitals-table">
<tbody>
<tr>
<th>Egg Groups</th>
<td>
<a href="/egg-group/grass">Grass</a>, <a href="/egg-group/monster">Monster</a> </td>
</tr>
<tr>
<th>Gender</th>
<td><span class="text-blue">87.5% male</span>, <span class="text-pink">12.5% female</span></td> </tr>
<tr>
<th><a href="/glossary#def-eggcycle">Egg cycles</a></th>
<td>20 <small class="text-muted">(4,884–5,140 steps)</small>
</td>
</tr>
</tbody>
</table>

In [80]:
table = page.findAll('table')[0] 
df = pd.read_html(str(table))[0]
pd.DataFrame([list(df[1])],columns = list(df[0]))

,National №,Type,Species,Height,Weight,Abilities,Local №
0,001,Grass Poison,Seed Pokémon,0.7 m (2′04″),6.9 kg (15.2 lbs),1. OvergrowChlorophyll (hidden ability),001 (Red/Blue/Yellow)226 (Gold/Silver/Crystal)...


In [86]:
table = page.findAll('table')[3] 
df = pd.read_html(str(table))[0]
pd.DataFrame([list(df[1])],columns = list(df[0]))

,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Total
0,45,49,49,65,65,45,318
